In [1]:
import xgi
import igraph as ig
import time
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
H_enron = xgi.load_xgi_data("plant-pollinator-mpl-034")
H_enron_cleaned = H_enron.cleanup(
    multiedges=False, singletons=False, isolates=False, relabel=True, in_place=False
)

In [3]:
print(H_enron)

# for node in H_enron.nodes:
#     print(f"Узел {node} принадлежит гиперребрам: {H_enron.nodes.memberships(node)}")

# xgi.drawing.draw(H_enron)
# plt.show()

Hypergraph named plant-pollinator-mpl-034 with 26 nodes and 128 hyperedges


In [4]:
xgi.convert.to_ig_line_graph(H_enron_cleaned)

In [15]:
start_time = time.time()

H_nx = xgi.convert.to_line_graph(H_enron)

middle_time = time.time()

H_ig = xgi.convert.to_ig_line_graph(H_enron)

end_time = time.time()

print(f"Время выполнения nx: {(middle_time-start_time):.6f} секунд")
print(f"Время выполнения ig: {(end_time-middle_time):.6f} секунд")

Время выполнения nx: 0.007002 секунд
Время выполнения ig: 0.002999 секунд


In [14]:
start_time = time.time()

cent_nx = nx.pagerank(H_nx)

middle_time = time.time()

cent_ig = H_ig.pagerank()

end_time = time.time()

print(f"Время выполнения nx: {(middle_time-start_time):.6f} секунд")
print(f"Посчитанная центральность nx: {cent_nx}")
print(f"Время выполнения ig: {(end_time-middle_time):.6f} секунд")
print(f"Посчитанная центральность ig: {cent_ig}")

Время выполнения nx: 0.006012 секунд
Посчитанная центральность nx: {'3': 0.020871119109190345, '4': 0.01843121571338235, '5': 0.014355970942220686, '6': 0.008291939562484012, '14': 0.003674435481659228, '18': 0.011756779975181753, '21': 0.005945594383753653, '28': 0.015712879410515686, '32': 0.003674435481659228, '33': 0.003674435481659228, '44': 0.003674435481659228, '55': 0.013721787649026082, '57': 0.012162839659696584, '64': 0.0066719271113430045, '68': 0.003674435481659228, '101': 0.014639687457844393, '125': 0.0074241034508579955, '0': 0.007490323475367359, '2': 0.014379054759029633, '8': 0.006405499932360633, '11': 0.008026897646011553, '12': 0.005849578100637959, '17': 0.007490323475367359, '20': 0.008026897646011553, '22': 0.007653083459395678, '31': 0.005849578100637959, '34': 0.005849578100637959, '43': 0.005849578100637959, '46': 0.013305921188617283, '53': 0.009851308336539761, '54': 0.012229259234055607, '58': 0.016430324139087116, '63': 0.005849578100637959, '65': 0.0058